In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
data = requests.get(team_urls[0])

In [8]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [9]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [22]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2,0,Crystal Palace,...,14.6,1.0,0,0,1.0,1.0,0.10,0.0,0.0,Match Report
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,2,Leicester City,...,13.0,0.0,0,0,2.7,2.7,0.16,1.3,1.3,Match Report
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3,0,Bournemouth,...,14.8,0.0,0,0,1.3,1.3,0.10,1.7,1.7,Match Report
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2,1,Fulham,...,15.5,1.0,0,0,2.6,2.6,0.12,-0.6,-0.6,Match Report
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2,1,Aston Villa,...,16.3,1.0,0,0,2.4,2.4,0.12,-0.4,-0.4,Match Report


In [ ]:
shooting.columns = shooting.columns.droplevel()

In [24]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [25]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-08-13,17:30,Premier League,Matchweek 2,Sat,Away,L,0,4,Brentford,...,4-2-3-1,Stuart Attwell,Match Report,NaN,19,7,13.0,0.0,0,0
1,2022-08-27,12:30,Premier League,Matchweek 4,Sat,Away,W,1,0,Southampton,...,4-2-3-1,Andy Madley,Match Report,NaN,22,8,15.5,1.0,0,0
2,2022-09-04,16:30,Premier League,Matchweek 6,Sun,Home,W,3,1,Arsenal,...,4-2-3-1,Paul Tierney,Match Report,NaN,16,3,18.6,1.0,0,0
3,2022-09-08,20:00,Europa Lg,Group stage,Thu,Home,L,0,1,es Real Sociedad,...,4-2-3-1,Marco Di Bello,Match Report,NaN,18,8,15.4,0.0,0,0
4,2022-10-06,19:45,Europa Lg,Group stage,Thu,Away,W,3,2,cy AC Omonia,...,4-2-3-1,João Pinheiro,Match Report,NaN,18,6,14.7,1.0,0,0


In [30]:
years = list(range(2022,2020,-1))
all_matches =[]

In [31]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [35]:
len(all_matches)


0

In [36]:
match_df = pd.concat(all_matches)

ValueError: No objects to concatenate

In [37]:
match_df.columns = [c.lower() for c in match_df.columns]

In [38]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,Match Report,NaN,13.0,1.0,17.8,0.0,0,0,2023,Liverpool
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,Match Report,NaN,25.0,9.0,16.8,1.0,0,1,2023,Liverpool
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,Match Report,NaN,9.0,4.0,17.2,1.0,0,0,2023,Liverpool
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,Match Report,NaN,17.0,4.0,14.7,0.0,0,0,2023,Liverpool
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,Match Report,NaN,16.0,5.0,15.8,0.0,0,0,2023,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,2023-05-02,20:00,Premier League,Matchweek 34,Tue,Home,W,3,1,Chelsea,...,Match Report,NaN,16.0,9.0,13.0,0.0,0,0,2022,Arsenal
45,2023-05-07,16:30,Premier League,Matchweek 35,Sun,Away,W,2,0,Newcastle Utd,...,Match Report,NaN,10.0,6.0,16.6,0.0,0,0,2022,Arsenal
46,2023-05-14,16:30,Premier League,Matchweek 36,Sun,Home,L,0,3,Brighton,...,Match Report,NaN,14.0,2.0,16.4,0.0,0,0,2022,Arsenal
47,2023-05-20,17:30,Premier League,Matchweek 37,Sat,Away,L,0,1,Nott'ham Forest,...,Match Report,NaN,11.0,3.0,16.4,0.0,0,0,2022,Arsenal


In [39]:
match_df.to_csv("matches.csv")